In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
# note scale = 2, or 3 makes it more precise, but too big.
scales = [1]
selected_columns = ['x', 'y']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side:
from zkstats.computation import computation_to_model, State

def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    return s.covariance(x, y)

error = 0.01
# Prover/ data owner side
_, prover_model = computation_to_model(computation, precal_witness_path, True, selected_columns, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)


/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/utils.py:1703: UserWarning: The exported ONNX model failed ONNX shape inf

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 1, param_scale: 1, scale_input_multiplier: 1) ------------->

+-------------+--------------+-----------+-------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error  | median_error | max_error | min_error   | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-------------+--------------+-----------+-------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.11755371 | -0.23510742  | 0         | -0.23510742 | 0.11755371     | 0.23510742       | 0.23510742    | 0             | 0.02763775         | -0.000036566813    | 0.000036566813         |
+-------------+--------------+-----------+-------------+----------------+------------------+---------------+----

scale:  [1]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":1,"param_scale":1,"scale_rebase_multiplier":1,"lookup_range":[-12560,12672],"logrows":15,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":13120,"total_assignments":5991,"total_const_size":2345,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,1],"model_input_scales":[1,1],"module_sizes":{"kzg":[],"poseidon":[13120,[2]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"required_range_checks":[[-1,1]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715154442549}


In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False, selected_columns, error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 3.589883327484131 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 3215.0
==== Generating Proof ====
proof:  {'instances': [['b8c4693b3424042141e265e5179743499536c8b9f1ddc7d4d4fc847c1c09df04', '7d055c183092753c601560dfbfa0b446f5417438227d99435c476a0659de3d14', '0100000000000000000000000000000000000000000000000000000000000000', '1e19000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x0660a66b41f0f766b4c61d69d379dac055e2ff25dc46fc6a7ee045e46009e0b128754ad917a3d2a456ae09bf1de7790a0b728ae0c903cba64568eed604e159990102a11efb5fbc168880ba84e916bf2473c7800d81b943e7f82623ac9bac04e32ac28b641cb160db41e7cc714c5c030c65ce520fe905ad6ddac6ac79431d02212e859f4a627be2fd6818b6ad72b825f55e4b71070b8a4c562f633eb25e3075f7106ae24e266e8ade883bc356696b1d89f163ff69685fb5b42b82fa031ce56c850ba46ac8801480cc005dd7089a096c234acf592141cb079f061472907159d1ac2b0cc0c795bbb410f92f976411facce7bb78b76d3c3f06850e6a63bfa72d2012284c53f75b

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [3215.0]
